# Dependencies and Configs

In [ ]:
pip install torch torchvision pyyaml

**Loading Configurations from the YAML file. [You may modify the file as your favour.]**

In [ ]:
import yaml

with open('../config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

# Preparing Train and Validation Datasets + Pre Processing

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from pathlib import Path

**Define Transforms**

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize the image to 224x224
    transforms.ToTensor(), # Convert the image to a pytorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalize the image to have pixel values in the range [-1, 1]
])

**Loading the Dataset**

In [ ]:
data_directory = Path(config['data_directory'])
dataset = datasets.ImageFolder(root=data_directory, transform=transform)

**Spiliting into Train and Test Datasets**

In [ ]:
train_size = int(0.85 * len(dataset))
test_size = len(dataset) - train_size
train, test = random_split(dataset, [train_size, test_size])
print(f'train size: {train_size}, test size: {test_size}')

**Defining Train and Test Dataloaders**

In [ ]:
batch_size = config['batch_size']
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [ ]:
id2label = {v: k for k, v in dataset.class_to_idx.items()}
label2id = dataset.class_to_idx
print(id2label)

# Training Model

**Load the pretrained ViT model**

In [ ]:
from timm import create_model
pre_trained_model = config['pre_trained_model']
model = create_model(pre_trained_model, pretrained=True, num_classes=len(train.dataset.classes))

**Moving model to GPU if available**

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available and will be used.")
else:
    device = torch.device("cpu")
    print("MPS is not available, using CPU.")

**Defining loss function and optimizer**

In [ ]:
import torch.nn as nn
import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()  # For classification
optimizer = optim.Adam(model.parameters(), lr=1e-4)

**Implementing the Early Stop Method**

In [ ]:
def early_stopping(val_loss, best_loss, patience_counter, patience, min_delta):
    if best_loss is None or val_loss < best_loss - min_delta:
        return val_loss, 0
    else:
        return best_loss, patience_counter + 1  # Increment counter if no improvement


**The Training Loop**

In [ ]:
from tqdm import tqdm  # For progress bar

In [ ]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)
print(device)

In [ ]:
num_epochs = config['num_epochs']
patience = config['patience']
learning_rate = config['learning_rate']

best_loss = None
patience_counter = 0

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    # Train over batches
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights

        running_loss += loss.item()

    # Print epoch loss
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation Loop
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Print validation metrics
    avg_val_loss = val_loss / len(test_loader)
    print(f'Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {100 * correct / total:.2f}%')

    # Early stopping check
    best_loss, patience_counter = early_stopping(avg_val_loss, best_loss, patience_counter, patience, learning_rate)

    if patience_counter >= patience:
        print("Early stopping triggered!")
        break


print("Training completed!")

**Saving the Model**

In [ ]:
# Save the trained model to a file
models_dir = '../models'
model_name = f'trained_{config["pre_trained_model"]}.pth'
model_path = Path(f'{models_dir}/{model_name}')
torch.save(model.state_dict(), model_path)
print(f"Model saved at {model_path}")